# Read result file 
    - meta data: available results, mesh, time/freq domain
    - results: displacement and stress

1st step : load dpf client/server, generate documentation if needed

In [20]:
from ansys import dpf
dpf.start_local_server()
#doc = dpf.Operator("html_doc")
#doc.inputs.output_path(r'c:/temp/doc.html')
#doc.run()
import os
res_files = os.getcwd() + r'\..\..\res_files'

server started at --address 10.28.1.60 --port 50052


2nd step : create a model

In [21]:
path = res_files + r'\file_static.rst'
model = dpf.Model(path)
metadata = model.metadata
results = model.results

In [22]:
print(metadata.result_info)

Static analysis
Unit system: Metric (m, kg, N, s, V, A)
Physics Type: Mecanic
Available results:
     -  displacement                  
     -  force                         
     -  stress                        
     -  volume                        
     -  energy_stiffness_matrix       
     -  hourglass_energy              
     -  thermal_dissipation_energy    
     -  kinetic_energy                
     -  co_energy                     
     -  incremental_energy            
     -  strain                        
     -  temperature                   



In [23]:
print(metadata.meshed_region)

Meshed Region
	7145 nodes
	4417 elements
	Unit: m 



In [24]:
print(metadata.time_freq_support)

Time/Frequency Info:
	Number of sets: 1

With complex values
 
 Cumulative      Time (s)       Loadstep     Substep   
     1             1.0             1            1      



## read displacements

In [25]:
displacements = results.displacement()

In [26]:
fields = displacements.outputs.fields_container()

In [27]:
disp_field = fields[0]

In [28]:
mesh = metadata.meshed_region
print(mesh)

Meshed Region
	7145 nodes
	4417 elements
	Unit: m 



In [29]:
print(disp_field)

DPF Displacement Field
	Location: Nodal
	Unit: m
	7145 id(s)
	data size: 21435
	shape: (7145, 3)



## read stresses

In [30]:
stress = results.stress()

In [31]:
stress.inputs.requested_location.connect("Nodal")

In [32]:
fields = stress.outputs.fields_container()

## export to vtk

In [33]:
vtk = dpf.Operator("vtk_export")

In [34]:
vtk.inputs.file_path(r'c:/temp/file.vtk')
vtk.inputs.mesh.connect(mesh)
vtk.inputs.fields1.connect(displacements.outputs.fields_container)
vtk.inputs.fields2.connect(stress.outputs.fields_container)
vtk.run()